# **Tool 1 - wikipedia**

In [4]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia_wrapper = WikipediaAPIWrapper()                       # wikipediaAPIWrapper - Just fetch Wikipedia results in your code
tool1= WikipediaQueryRun(api_wrapper=wikipedia_wrapper)         # WikipediaQueryRun (tool) - Give an agent the ability to search Wikipedia

search = tool1.invoke("what is python")
search


'Page: Python (programming language)\nSummary: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.\nPython is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.\nGuido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language, and he first released it in 1991 as Python 0.9.0. Python 2.0 was released in 2000. Python 3.0, released in 2008, was a major revision not completely backward-compatible with earlier versions. Python 2.7.18, released in 2020, was the last release of Python 2.\nPython consistently ranks as one of the most popular programming languages, and it has gained widespread use in the machine learning comm

# **Tool 2 - arxiv**

In [7]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper()
tool2= ArxivQueryRun(api_wrapper=arxiv_wrapper)

search = tool2.invoke("what is python")
search

"Published: 2023-06-14\nTitle: Characterizing Bugs in Python and R Data Analytics Programs\nAuthors: Shibbir Ahmed, Mohammad Wardat, Hamid Bagheri, Breno Dantas Cruz, Hridesh Rajan\nSummary: R and Python are among the most popular languages used in many critical data\nanalytics tasks. However, we still do not fully understand the capabilities of\nthese two languages w.r.t. bugs encountered in data analytics tasks. What type\nof bugs are common? What are the main root causes? What is the relation between\nbugs and root causes? How to mitigate these bugs? We present a comprehensive\nstudy of 5,068 Stack Overflow posts, 1,800 bug fix commits from GitHub\nrepositories, and several GitHub issues of the most used libraries to\nunderstand bugs in R and Python. Our key findings include: while both R and\nPython have bugs due to inexperience with data analysis, Python see\nsignificantly larger data preprocessing bugs compared to R. Developers\nexperience significantly more data flow bugs in R b

# **Tool 3 - VectorDB**

In [25]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

web_loader =WebBaseLoader("https://python.langchain.com/docs/introduction/")
web_docs = web_loader.load()

text_splitter = RecursiveCharacterTextSplitter()
web_chunks = text_splitter.split_documents(web_docs)

#web_chunks = RecursiveCharacterTextSplitter().split_documents(web_docs)

embedding_model = OllamaEmbeddings(model="nomic-embed-text:latest")

DB = FAISS.from_documents(web_chunks,embedding=embedding_model)

retriver = DB.as_retriever()

In [60]:
from langchain.tools.retriever import create_retriever_tool

tool3 = create_retriever_tool(retriever=retriver , name="Langchain_search",
                              description="For any question asked regarding the langchain use this tool ")

tool3

Tool(name='Langchain_search', description='For any question asked regarding the langchain use this tool ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000019227850EA0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000192B199F470>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000019227850FE0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000192B199F470>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_c

# **Load LLM**

In [54]:
from langchain.prompts import ChatPromptTemplate , MessagesPlaceholder
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate


promptTemplate = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "You are an AI assistant with tools integrated to answer the query. "
        "Use the tools provided to fetch relevant information before giving a generalized answer. "
        "Make sure the response is accurate and backed by facts and relevant sources."
    ),
    HumanMessagePromptTemplate.from_template("{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),  # Required for tool use
])




In [61]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="openai/gpt-4o-mini",
                 base_url="https://openrouter.ai/api/v1",
                 api_key="sk-or-v1-0ac87964a3d91baxxxxxxxxxxe020b56e4181fb7cf0ab8xxxxxxxxxxf8d200c7")


tools= [tool1 ,tool2,tool3 ]



# **Create Agent with tools**

In [62]:
from langchain.agents  import create_openai_tools_agent

Agent = create_openai_tools_agent(llm,tools,promptTemplate)

# **Create Agent Executor**

In [71]:
from langchain.agents import AgentExecutor

excecutor = AgentExecutor.from_agent_and_tools(Agent,tools,verbose=True)

result=excecutor.invoke({"input":"what is the paper 1605 is about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605'}`



Invoking: `arxiv` with `{'query': '1605'}`


Published: 2023-10-31
Title: CCD UBV and Gaia DR3 Analyses of Open Clusters King 6 and NGC 1605
Authors: S. Gokmen, Z. Eker, T. Yontan, S. Bilir, T. Ak, S. Ak, T. Banks, A. Sarajedini
Summary: A detailed analysis of ground-based CCD UBV photometry and space-based Gaia
Data Release 3 (DR3) data for the open clusters King 6 and NGC 1605 was
performed. Using the pyUPMASK algorithm on Gaia astrometric data to estimate
cluster membership probabilities, we have identified 112 stars in King 6 and
160 stars in NGC 1605 as the statistically most likely members of each cluster.
We calculated reddening and metallicity separately using UBV two-color diagrams
to estimate parameter values via independent methods. The color excess $E(B-V)$
and photometric metallicity [Fe/H] for King 6 are $0.515 \pm 0.030$ mag and
$0.02 \pm 0.20$ dex, respectively. For NGC 1605, they are

In [72]:
result["output"]

'The paper you are referring to in arXiv with the identifier "1605" is actually a collection of multiple papers published in May 2016. Here\'s a summary of notable ones:\n\n1. **Title:** On the number of cyclic subgroups in finite groups\n   - **Authors:** Wei Zhou\n   - **Summary:** This paper studies the number of cyclic subgroups in finite groups and concludes that a group \\( G \\) has \\( |G|-3 \\) cyclic subgroups if and only if \\( G \\) is isomorphic to either the dihedral group \\( D_{10} \\) or the quaternion group \\( Q_8 \\).\n\n2. **Title:** On the Roots of Characteristic Equations of Delay Differential Systems\n   - **Authors:** Jia-Yuan Dai\n   - **Summary:** This paper proves that the characteristic equations of specific types of delay differential systems can possess infinitely many complex roots under certain mild conditions on their coefficients.\n\nAdditionally, a more recent paper published in October 2023 investigates two open clusters, King 6 and NGC 1605, using 